In [18]:
import wandb
wandb.login()

wandb: Currently logged in as: sharathhebbar to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [19]:
wandb.init(
    # set the wandb project where this run will be logged
    project="sft_smollm",
)

In [20]:
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [21]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [ ]:

dataset_name = "HuggingFaceTB/smoltalk"
subset = "everyday-conversations"
dataset = load_dataset(dataset_name, subset)
dataset

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

In [23]:
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)


model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

In [ ]:
data = dataset["train"].shuffle().map(generate_and_tokenize_prompt)

In [26]:
trainings_args = SFTConfig(
    output_dir="./sft_output",
    max_steps=200,
    per_device_train_batch_size=4,
    learning_rate=5e-5,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=50
)

In [27]:
trainer = SFTTrainer(
    model=model,
    args=trainings_args,
    train_dataset=dataset['train'],
    eval_dataset=dataset['test'],
    processing_class=tokenizer
)

trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss
50,1.070600,1.164202
100,1.113100,1.124792
150,1.067200,1.103564
200,1.060500,1.095357


TrainOutput(global_step=200, training_loss=1.1416414737701417, metrics={'train_runtime': 658.9996, 'train_samples_per_second': 1.214, 'train_steps_per_second': 0.303, 'total_flos': 116030249376768.0, 'train_loss': 1.1416414737701417})

In [28]:
import os
from dotenv import load_dotenv
load_dotenv()
MODEL_PATH = os.getenv("MODEL_PATH")
HF_TOKEN = os.getenv("HF_TOKEN")

In [31]:
tokenizer.push_to_hub(
    MODEL_PATH,
    token=HF_TOKEN
)

CommitInfo(commit_url='https://huggingface.co/Sharathhebbar24/smollm_sft/commit/62a80d5c2d48d58121cdccad8e1a7a2b1eccd370', commit_message='Upload tokenizer', commit_description='', oid='62a80d5c2d48d58121cdccad8e1a7a2b1eccd370', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Sharathhebbar24/smollm_sft', endpoint='https://huggingface.co', repo_type='model', repo_id='Sharathhebbar24/smollm_sft'), pr_revision=None, pr_num=None)

In [32]:
model.push_to_hub(
    MODEL_PATH,
    token=HF_TOKEN
)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\sharath\Github\Projects\Loan prediction\loan-pred\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shash\.cache\huggingface\hub\models--Sharathhebbar24--smollm_sft. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/538M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sharathhebbar24/smollm_sft/commit/0600af61fa6e1be39a4089940656e1f4ab419a17', commit_message='Upload LlamaForCausalLM', commit_description='', oid='0600af61fa6e1be39a4089940656e1f4ab419a17', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Sharathhebbar24/smollm_sft', endpoint='https://huggingface.co', repo_type='model', repo_id='Sharathhebbar24/smollm_sft'), pr_revision=None, pr_num=None)

In [1]:
import gc
gc.collect()

10

In [2]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
from transformers import BitsAndBytesConfig

config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

In [6]:
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer, setup_chat_format
dataset_name = "HuggingFaceTB/smoltalk"
subset = "everyday-conversations"
dataset = load_dataset(dataset_name, subset)
dataset

DatasetDict({
    train: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 2260
    })
    test: Dataset({
        features: ['full_topic', 'messages'],
        num_rows: 119
    })
})

In [22]:
len(dataset['train']['messages'])

2260

In [7]:
from transformers import (
    AutoModelForCausalLM, AutoTokenizer
)
model_name = "HuggingFaceTB/SmolLM2-135M"

model = AutoModelForCausalLM.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)


model, tokenizer = setup_chat_format(model=model, tokenizer=tokenizer)

In [32]:
# def generate_prompt(data_point):
#    return data_point['messages']
# def generate_and_tokenize_prompt(data_point):
#     full_prompt = generate_prompt(data_point)
#     tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
#     return tokenized_full_prompt

# data = dataset["train"].shuffle().map(generate_and_tokenize_prompt)
# data

In [27]:
from peft import prepare_model_for_kbit_training

quantized_model = prepare_model_for_kbit_training(model)

In [28]:
from peft import LoraConfig

rank_dimension = 16
lora_alpha = 8
# target_modules=["q_proj", "k_proj", "v_proj", "o_proj"]
lora_dropout = 0.05
peft_config = LoraConfig(
    r=rank_dimension,
    lora_alpha=lora_alpha,
    # target_modules=target_modules,
    lora_dropout=lora_dropout,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM"
)

In [29]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [30]:
from peft import get_peft_model

quantized_model = get_peft_model(quantized_model, peft_config)


c:\sharath\Github\Projects\Loan prediction\loan-pred\.venv\Lib\site-packages\peft\mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
c:\sharath\Github\Projects\Loan prediction\loan-pred\.venv\Lib\site-packages\peft\tuners\tuners_utils.py:167: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [31]:
print_trainable_parameters(quantized_model)

trainable params: 4884480 || all params: 139399488 || trainables%: 3.503944003008103


In [33]:
import os
from dotenv import load_dotenv
load_dotenv()
MODEL_PATH = os.getenv("MODEL_PATH")
HF_TOKEN = os.getenv("HF_TOKEN")

In [34]:
tokenizer.save_pretrained("./smollm_quantized_peft")

('./smollm_quantized_peft\\tokenizer_config.json',
 './smollm_quantized_peft\\special_tokens_map.json',
 './smollm_quantized_peft\\vocab.json',
 './smollm_quantized_peft\\merges.txt',
 './smollm_quantized_peft\\added_tokens.json',
 './smollm_quantized_peft\\tokenizer.json')

In [35]:
quantized_model.save_pretrained("./smollm_quantized_peft")


In [38]:
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)

PEFT_MODEL = "./smollm_quantized_peft"
q_model_config = PeftConfig.from_pretrained(PEFT_MODEL)
q_model = AutoModelForCausalLM.from_pretrained(
    q_model_config.base_model_name_or_path,
    return_dict=True,
    quantization_config=config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(q_model_config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

q_model = PeftModel.from_pretrained(q_model, PEFT_MODEL)

In [39]:
MODEL_PATH = "Sharathhebbar24/smollm_peft"
tokenizer.push_to_hub(
    MODEL_PATH,
    token=HF_TOKEN
)
q_model.push_to_hub(
    MODEL_PATH,
    token=HF_TOKEN
)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

c:\sharath\Github\Projects\Loan prediction\loan-pred\.venv\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\shash\.cache\huggingface\hub\models--Sharathhebbar24--smollm_peft. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


adapter_model.safetensors:   0%|          | 0.00/19.6M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Sharathhebbar24/smollm_peft/commit/7042c126475ee54b45cba51e52837882d969459f', commit_message='Upload model', commit_description='', oid='7042c126475ee54b45cba51e52837882d969459f', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Sharathhebbar24/smollm_peft', endpoint='https://huggingface.co', repo_type='model', repo_id='Sharathhebbar24/smollm_peft'), pr_revision=None, pr_num=None)

In [36]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, GPTQConfig

# model_id = "facebook/opt-125m"
# tokenizer = AutoTokenizer.from_pretrained(model_id)

# gptq_config = GPTQConfig(bits=4, group_size=128, dataset="wikitext2", tokenizer=tokenizer)

# quantized_model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", quantization_config=gptq_config)

# # save quantized model
# # quantized_model.save_pretrained("./opt-125m-gptq")
# # tokenizer.save_pretrained("./opt-125m-gptq")

In [40]:
torch.cuda.empty_cache()